# Intro to Data Science
## Part VI. - Model Evaluation, Hyperparameter optimization, Clustering

### Table of contents

- #### Model evaluation
    - <a href="#What-is-Model-Evaluation?">Theory</a>
    - <a href="#1.-Classification-metrics">Classification Metrics</a>
        - <a href="#a)-Accuracy">Accuracy</a>
        - <a href="#b)-Confusion-matrix">Confusion matrix</a>
        - <a href="#c)-Precision,-recall,-f1-score">Precision, Recall, F1 score</a>
        - <a href="#d)-ROC-curve">ROC curve</a>
    - <a href="#2.-Regression-metrics">Regression Metrics</a>
        - <a href="#a)-Explained-variance-score">Explained variance</a>
        - <a href="#b)-Mean-absolute-error-(MAE)">MAE</a>
        - <a href="#c)-Mean-squared-error-(MSE)">MSE</a>
    - <a href="#Cluster-Validation">Clustering Metrics</a>
    
- #### Hyperparameter optimization
    - <a href="#What-is-Hyperparameter-optimization?">Theory</a>
    - <a href="#Cross-Validation">Cross Validation</a>
        - <a href="#1.-Grid-Search-Cross-Validation">Grid Search Cross Validation</a>
        - <a href="#2.-Randomized-Search-Cross-Validation">Randomized Search Cross Validation</a>
    - <a href="#3.-Other-Hyperparameter-searching-methods">Other Hyperparameter searching methods</a>
        
- #### Clustering
    - <a href="#What-is-Clustering?">Theory</a>
    - Clustering methods
        - <a href="#1.-K-Means">K-means</a>
        - <a href="#2.-DBSCAN">DBSCAN</a>
        - <a href="#3.-Hierarchical-Clustering">Hierarchical clustering</a>
        - <a href="#4.-Spectral-Clustering">Spectral clustering</a>
        - <a href="#5.-Gaussian-Mixture-Models">Gaussian Mixture Models</a>
    
---

# I. Model Evaluation

## What is Model Evaluation?

Working with machine learning algorithms, data mining techniques or simple statistical models requires a proper indication if a model is trained properly. Depending on the task, there are several metrics available to estimate the goodness of a fitted model. Beside raw metrics there are some important concepts on how to compare models. Some of them shows if a model is overfitted, some can show if a model is simpler than the other.

## Why is it important?

To find the optimal solution for a given problem it is crucial to have an insight about the performance of the proposed models to decide whether to continue the training process, try a different preprocessing pipeline, or a different model.

## Tools
- Classification metrics
    - accuracy
    - precision
    - recall
    - precision-recall curve
    - f1 score
    - confusion matrix
    - ROC
- Regression metrics
    - mean absolute error
    - RMSE
    - explained variance score
- Clustering metrics
- Cross Validiation
- etc.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline

from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

np.random.seed = 42

In [ ]:
X_dig, y_dig = load_digits(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X_dig, y_dig,
                                                    test_size=.25,
                                                    random_state=42)

In [ ]:
nn_pipe = Pipeline([('nn', MLPClassifier(hidden_layer_sizes=(5,), random_state=42))])
nn_pipe.fit(X_train, y_train)
y_hat = nn_pipe.predict(X_test)

## 1. Classification metrics

### a) [Accuracy](http://scikit-learn.org/stable/modules/model_evaluation.html#accuracy-score)
Accuracy is the ratio of correct prediction and defined by
$$\texttt{accuracy}(y, \hat{y}) = \frac{1}{n_\text{samples}} \sum_{i=0}^{n_\text{samples}-1} 1(\hat{y}_i = y_i)$$
where $1(x)$ is the indicator function.

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_test, y_hat)

### b) Confusion matrix

A [confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix#Table_of_confusion) shows the number of correct and incorrect predictions made by the classification model compared to the actual outcomes (target value) in the data.


In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test, y_hat)

In [ ]:
sns.heatmap(confusion_matrix(y_test, y_hat))

### c) Precision, recall, f1 score

_"Intuitively, precision is the ability of the classifier not to label as positive a sample that is negative, and recall is the ability of the classifier to find all the positive samples."_ from [sklearn docs](http://scikit-learn.org/stable/modules/model_evaluation.html#precision-recall-and-f-measures)

Precision, recall and F1 score uses the notation `true positive`, `true negative`, `false positive` and `false negative`. These values represents the 4 possible outcome from a binary classification problem and can easily understand through the following confusion matrix:


<table border="1" width="620" bordercolor="#C0C0C0" cellspacing="0" bordercolorlight="#C0C0C0" bordercolordark="#FFFFFF">
    <tbody><tr>
      <td align="center" width="170" colspan="2" rowspan="2"><font face="Calibri">Confusion Matrix</font></td>
      <td align="center" width="191" colspan="2" bgcolor="#E3EBF2"><font face="Calibri"><b>Target</b></font></td>
    </tr>
    <tr>
      <td align="center" width="92" bgcolor="#E3EBF2"><font face="Calibri">Positive</font></td>
      <td align="center" width="95" bgcolor="#E3EBF2"><font face="Calibri">Negative</font></td>
    </tr>
    <tr>
      <td rowspan="2" align="center" width="94" bgcolor="#E3EBF2"><font face="Calibri"><b>Model</b></font></td>
      <td align="center" width="72" bgcolor="#E3EBF2"><font face="Calibri">Positive</font></td>
      <td align="center" width="92"><font face="Calibri">a</font></td>
      <td align="center" width="95"><font face="Calibri">b</font></td>
    </tr>
    <tr>
      <td align="center" width="72" bgcolor="#E3EBF2"><font face="Calibri">Negative</font></td>
      <td align="center" width="92"><font face="Calibri">c</font></td>
      <td align="center" width="95"><font face="Calibri">d</font></td>
    </tr>
  </tbody></table>


- __[Precision](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html#sklearn.metrics.precision_score):__
The precision is the ratio `tp / (tp + fp)` where `tp` is the number of true positives and `fp` the number of false positives. The precision is intuitively the ability of the classifier not to label as positive a sample that is negative.

- __[Recall](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html#sklearn.metrics.recall_score):__
The recall is the ratio `tp / (tp + fn)` where `tp` is the number of true positives and `fn` the number of false negatives. The recall is intuitively the ability of the classifier to find all the positive samples.

- __[F1 score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score):__
The F1 score can be interpreted as a weighted average of the precision and recall, where an F1 score reaches its best value at 1 and worst score at 0. The relative contribution of precision and recall to the F1 score are equal. The formula for the F1 score is:
`F1 = 2 * (precision * recall) / (precision + recall)`
In the multi-class and multi-label case, this is the weighted average of the F1 score of each class.

These metrics are designed to work with binary classification problems, however there are several strategy to compute the multilabel variant as well. One can either compute the mean of the precision|recall|f1 scores (`average='macro'`), or use the unions of the `tp`, `tn`, `fp`, `fn` values to compute the metrics (`average='micro'`). You can read about the different strategies [here](http://scikit-learn.org/stable/modules/model_evaluation.html#multiclass-and-multilabel-classification).

In [ ]:
from sklearn.metrics import precision_score
precision_score(y_test, y_hat, average=None)

In [ ]:
from sklearn.metrics import recall_score
recall_score(y_test, y_hat, average=None)

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, y_hat, average=None)

### d) [ROC curve](http://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html)

_“A receiver operating characteristic (ROC), or simply ROC curve, is a graphical plot which illustrates the performance of a binary classifier system as its discrimination threshold is varied. It is created by plotting the fraction of true positives out of the positives (TPR = true positive rate) vs. the fraction of false positives out of the negatives (FPR = false positive rate), at various threshold settings. TPR is also known as sensitivity, and FPR is one minus the specificity or true negative rate.”_ from [sklearn user guide](https://en.wikipedia.org/wiki/Receiver_operating_characteristic)

The most important value to extract from this graph is the __area under the curve (AUC)__ value, which _"is equal to the probability that a classifier will rank a randomly chosen positive instance higher than a randomly chosen negative one"_ from [wiki](https://en.wikipedia.org/wiki/Receiver_operating_characteristic#Area_under_the_curve)

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

Since ROC can only be interpreted in binary classification, we have to transform our multilabel data into the required format.  
For this we have to:

- generate prediction probability for each class

In [ ]:
y_score = nn_pipe.predict_proba(X_test)
classes = np.unique(y_dig)

- transform labels into binary classes (1 for the current class, 0 otherwise)

In [ ]:
def onevsrest(array, label):
    return (array == label).astype(int)

- compute __TPR__, __FPR__ and __AUC__ for each class
>    `roc_curve` returns the __FPR__, __TPR__ and the __threshold__ arrays

In [ ]:
fpr, tpr, thres, roc_auc = {}, {}, {}, {}
for i in classes:
    fpr[i], tpr[i], thres[i] = roc_curve(onevsrest(y_test, i), y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

- plot all of the curves into one axis

In [ ]:
mycolors = sns.color_palette('muted', n_colors=len(classes))
fig, ax = plt.subplots(figsize=(8, 8))

ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')

# plot ROC for random baseline classifier
ax.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')

# plot ROC for each class
for cls in classes:
    label = (f'ROC curve for {cls} (area = {roc_auc[cls]:0.2f})')

    ax.plot(fpr[cls], tpr[cls], color=mycolors[cls], lw=2, label=label)
    ax.legend(loc="lower right")

## 2. Regression metrics

In [ ]:
reg_pipe = Pipeline([('reg', LogisticRegression(solver='liblinear', multi_class='auto', random_state=42))])
reg_pipe.fit(X_train, y_train)
y_hat = reg_pipe.predict(X_test)

### a) [Explained variance score](http://scikit-learn.org/stable/modules/model_evaluation.html#explained-variance-score)

If $\hat{y}$ is the estimated target output, $y$ the corresponding (correct) target output, and $Var$ is Variance, the square of the standard deviation, then the explained variance is estimated as follow:
$$\texttt{explained_variance}(y, \hat{y}) = 1 - \frac{Var\{ y - \hat{y}\}}{Var\{y\}}$$

In [ ]:
from sklearn.metrics import explained_variance_score
explained_variance_score(y_test, y_hat)

### b) [Mean absolute error (`MAE`)](http://scikit-learn.org/stable/modules/model_evaluation.html#mean-absolute-error)

MAE is a risk metric corresponding to the expected value of the absolute error loss or $l1$-norm loss.  
If $\hat{y}_i$ is the predicted value of the $i$-th sample, and $y_i$ is the corresponding true value, then the mean absolute error (MAE) estimated over $n_{\text{samples}}$ is defined as
$$\text{MAE}(y, \hat{y}) = \frac{1}{n_{\text{samples}}} \sum_{i=0}^{n_{\text{samples}}-1} \left| y_i - \hat{y}_i \right|.$$

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, y_hat)

### c) [Mean squared error (`MSE`)](http://scikit-learn.org/stable/modules/model_evaluation.html#mean-squared-error)

MSE is a risk metric corresponding to the expected value of the squared (quadratic) error loss or loss.  
If $\hat{y}_i$ is the predicted value of the i-th sample, and $y_i$ is the corresponding true value, then the mean squared error (MSE) estimated over $n_{\text{samples}}$ is defined as
$$\text{MSE}(y, \hat{y}) = \frac{1}{n_\text{samples}} \sum_{i=0}^{n_\text{samples} - 1} (y_i - \hat{y}_i)^2.$$

It's widely used variant, the __Root Mean Squared Error (`RMSE`)__ is computed by getting the root of MSE.

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_hat)

---
# II. Hyperparameter optimization

## What is Hyperparameter optimization?

According to [wikipedia](https://en.wikipedia.org/wiki/Hyperparameter_optimization):  
_"In the context of machine learning, __hyperparameter optimization__ or __model selection__ is the problem of choosing a set of hyperparameters for a learning algorithm, usually with the goal of optimizing a measure of the algorithm's performance on an independent data set. Often cross-validation is used to estimate this generalization performance.  
Hyperparameter optimization contrasts with actual learning problems, which are also often cast as optimization problems, but optimize a loss function on the training set alone. In effect, learning algorithms learn parameters that model/reconstruct their inputs well, while hyperparameter optimization is to ensure the model does not overfit its data by tuning, e.g., regularization."_

## Why is it important?

To find the optimal solution to a given problem, one must train several models with similar predictive/exploratory power and select the simplest one. This process includes selecting models and finding optimal hyperparameters which is a time consuming and tedious work when done by hand. We use automatized solutions to overcome this problem, save time, and yield better results.

## Tools
- Grid search
- Randomized search
- Bayesian search
- Gradient-base optimization 
- TPOT
- etc.

---

## <a href="http://scikit-learn.org/stable/modules/cross_validation.html#cross-validation">Cross Validation</a>

In order to select an optimal model, first one must be able to measure a model's/pipe's accuracy.  

First, one must select a valid metric for the model. In sklearn, the basic validation metric is accuracy score in case of classification, and $r^{2}$ for regression. Altough, several other metrics can be selected from <a href="http://scikit-learn.org/stable/modules/model_evaluation.html#the-scoring-parameter-defining-model-evaluation-rules">this</a> list.

_"Learning the parameters of a prediction function and testing it on the same data is a methodological mistake: a model that would just repeat the labels of the samples that it has just seen would have a perfect score but would fail to predict anything useful on yet-unseen data. This situation is called overfitting."_ [1]. To overcome this problem, one must split the data to __training__ and __test__ dataset; train the model on the train dataset, then measure the precision on the test dataset.

However different splits can produce different outcomes, so this process must be repeated several times to give a good approximation to the examined model's accuracy. This process is called __Cross Validation__ and there are <a href="http://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-iterators">different strategies</a> to make these splits.

A simple model can yield different solutions to the same data based on its hyperparameters so multiple models must be trained to select the ideal hyperparamter settings. Cross Validation gives a good approximation to a trained model's accuracy, but additional methods are required to select the ideal hyperparameters. 

[1] <a href="http://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-evaluating-estimator-performance">Scikit-learn User Guide</a>

### 1. Grid Search Cross Validation
Grid search is a method which generates a parameter grid from a list of settings, and measure the input model's accuracy in every setting using cross validation.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV

In [ ]:
pipe_digit = Pipeline([
    ('pca', PCA(svd_solver='randomized')),
    ('logistic', LogisticRegression(solver='liblinear', multi_class='auto', random_state=42))
])

In [ ]:
param_grid = {
    'pca__n_components': [20, 40, 64],
    'logistic__C': np.logspace(-4, 4, 3)
}

In [ ]:
grid_search = GridSearchCV(estimator=pipe_digit, 
                           param_grid=param_grid,
                           n_jobs=-1, 
                           iid=True, 
                           cv=5,
                           verbose=1, 
                           return_train_score=True)

In [ ]:
grid_search.fit(X_dig, y_dig)

In [ ]:
grid_search.best_estimator_.get_params(deep=False)

In [ ]:
grid_search.best_params_, grid_search.best_score_

In [ ]:
grid_search.cv_results_

In [ ]:
score_dict = grid_search.cv_results_
hmap = pd.DataFrame({
    'mean': score_dict['mean_test_score'],
    'C': [param['logistic__C'] for param in score_dict['params']],
    'n': [param['pca__n_components'] for param in score_dict['params']]
})

In [ ]:
sns.heatmap(hmap.pivot(index='C', columns='n', values='mean'), annot=True, fmt='.3f')

### 2. Randomized Search Cross Validation
Randomized search randomly generate a fixed number of hyperparameter setups. It selects the parameters from the provided parameter parameter ranges and then measures them with cross validation.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
random_search_digit = RandomizedSearchCV(
    pipe_digit,
    {
        'pca__n_components': np.linspace(10, 64, 91, dtype=int),
        'logistic__C': np.logspace(-4, 4, 30),
    },
    n_iter=30, 
    n_jobs=-1,
    iid=True,
    cv=5,
    verbose=1,
    return_train_score=True)

In [ ]:
random_search_digit.fit(X_dig, y_dig)

In [ ]:
random_score_dict = random_search_digit.cv_results_
hmap_r = pd.DataFrame({
    'mean': random_score_dict['mean_test_score'],
    'C': [param['logistic__C'] for param in random_score_dict['params']],
    'n': [param['pca__n_components'] for param in random_score_dict['params']]
})

In [ ]:
fig, ax = plt.subplots(figsize=(12,10))
sns.heatmap(hmap_r.pivot(index='C', columns='n', values='mean'), annot=True, ax=ax)

### 3. Other Hyperparameter searching methods

- <a href="https://github.com/rhiever/tpot">TPOT</a>
- <a href="https://www.automl.org/automl/auto-sklearn/">auto-sklearn</a>
- <a href="http://hyperopt.github.io/hyperopt/">hyperopt</a>
- <a href="https://medium.com/rants-on-machine-learning/smarter-parameter-sweeps-or-why-grid-search-is-plain-stupid-c17d97a0e881#.aify5h22n">Other hyperparameter searching methods</a> 

---
# III. Clustering

## What is Clustering?
Clustering is an <a href="http://scikit-learn.org/stable/unsupervised_learning.html">unsupervised machine learning</a> problem. _"Unsupervised learning is the machine learning task of inferring a function to describe hidden structure from unlabeled data. Since the examples given to the learner are unlabeled, there is no error or reward signal to evaluate a potential solution."_ from: <a href="https://en.wikipedia.org/wiki/Unsupervised_learning">Wiki</a> 

_"Cluster analysis or clustering is the task of grouping a set of objects in such a way that objects in the same group (called a cluster) are more similar (in some sense or another) to each other than to those in other groups (clusters)."_ from: <a href="https://en.wikipedia.org/wiki/Cluster_analysis">Wiki</a>


## Why is it important?
Often the data does not contain target variables so one must find the hidden structure in the data first in order to achieve his/her goals. In case of recommender systems, it is a common technique to group the similar items together. In some cases the task itself is to find similar/connected/related items in the data. Like in image processing, Social network analysis, medical analysis, or it can be used to find the anomalies in the data.

## Tools
- K-Means
- Affinity propagation
- Mean-shift
- Spectral clustering
- Ward hierarchical clustering
- Agglomerative clustering
- DBSCAN
- Gaussian mixtures
- Birch
- Support Vector Clustering
- etc.

In [ ]:
from sklearn.datasets import make_circles
from sklearn.datasets import make_moons
from sklearn.datasets import make_blobs

In [ ]:
n_clusters = 3
n_samples = 1500

noisy_circles = make_circles(n_samples=n_samples, factor=.5, noise=.05, random_state=42)
noisy_moons = make_moons(n_samples=n_samples, noise=.05, random_state=42)
blobs = make_blobs(n_samples=n_samples, random_state=42)
no_structure = np.random.rand(n_samples, 2), None

datasets = {
    'noisy_circles': noisy_circles,
    'noisy_moons': noisy_moons,
    'blobs': blobs,
    'no_structure': no_structure
}

colors = np.array(sns.color_palette('muted', n_colors=5))

In [ ]:
def cluster_datasets(model, preprocess=None, **params):
    model = model(**params)
    results = {}
    Xs = {}
    for problem, dataset in datasets.items():
        X, y = dataset
        if preprocess:
            X = preprocess.fit_transform(X, y)
        Xs[problem] = X
        model.fit(X)
        if hasattr(model, 'labels_'):
            results[problem] = model.labels_.astype(np.int)
        else:
            results[problem] = model.predict(X)
    return model, Xs, results

In [ ]:
def plot(Xs, results):
    plot_num = 1
    plt.figure(figsize=(len(datasets) * 4, 4))
    for problem, X in Xs.items():
        plt.subplot(1, len(datasets), plot_num)
        plt.scatter(X[:, 0], X[:, 1], color=colors[results[problem]], edgecolors='k')
        plot_num += 1

## 1. [K-Means](http://scikit-learn.org/stable/modules/clustering.html#k-means)


K-Means clustering intends to partition n objects into k clusters in which each object belongs to the cluster with the nearest mean. This method produces exactly k different clusters of greatest possible distinction. The best number of clusters k leading to the greatest separation (distance) is not known as a priori and must be computed from the data. The objective of K-Means clustering is to minimize total intra-cluster variance, or, the squared error function: 

![K-means target function](http://www.saedsayad.com/images/Clustering_kmeans_c.png)

__Algorithm__:
1. Clusters the data into k groups where k  is predefined.
2. Select k points at random as cluster centers.
3. Assign objects to their closest cluster center according to the Euclidean distance function.
4. Calculate the centroid or mean of all objects in each cluster.
5. Repeat steps 2, 3 and 4 until the same points are assigned to each cluster in consecutive rounds.

from: Dr. Saed Sayad's [__An Introduction to Data Mining__](http://www.saedsayad.com/clustering_kmeans.htm) book

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, MiniBatchKMeans

In [ ]:
model, Xs, results = cluster_datasets(
    model=KMeans,
    preprocess=StandardScaler(),
    n_clusters=3,
    random_state=42
)
plot(Xs, results)

In [ ]:
model, Xs, results = cluster_datasets(
    MiniBatchKMeans,
    preprocess=StandardScaler(),
    n_clusters=3,
    random_state=42
)
plot(Xs, results)

## 2. [DBSCAN](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html)

*"The __Density-based spatial clustering of applications with noise (DBSCAN)__ algorithm views __clusters__ as __areas of high density separated by areas of low density__. Due to this rather generic view, clusters found by DBSCAN can be any shape, as opposed to k-means which assumes that clusters are convex shaped. The central component to the DBSCAN is the concept of __core samples__, which are samples that are in areas of high density. A cluster is therefore a set of core samples, each close to each other (measured by some distance measure) and a set of __non-core samples__ () that are close to a core sample (but are not themselves core samples). There are two parameters to the algorithm, `min_samples` and `eps`, which define formally what we mean when we say dense. Higher `min_samples` or lower `eps` indicate higher density necessary to form a cluster."* from sklearn's [User Guide](http://scikit-learn.org/stable/modules/clustering.html#dbscan).

Consider a set of points in some space to be clustered. For the purpose of DBSCAN clustering, the points are classified as __core points__, (density-)__reachable points__ and __outliers__, as follows:

- A point $p$ is a core point if at least $minPts$ points are within distance $ε$ ($ε$ is the maximum radius of the neighborhood from $p$) of it (including $p$). Those points are said to be directly reachable from $p$. By definition, no points are directly reachable from a non-core point.
- A point $q$ is reachable from $p$ if there is a path $p_1, ..., p_n$ with $p_1 = p$ and $p_n = q$, where each $p_i+1$ is directly reachable from $p_i$ (all the points on the path must be core points, with the possible exception of $q$).
- All points not reachable from any other point are outliers.

Now if $p$ is a core point, then it forms a cluster together with all points (core or non-core) that are reachable from it. Each cluster contains at least one core point; non-core points can be part of a cluster, but they form its "edge", since they cannot be used to reach more points. from [wiki](https://en.wikipedia.org/wiki/DBSCAN#Preliminary)


In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
model, Xs, results = cluster_datasets(
    DBSCAN,
    preprocess=StandardScaler(),
    eps=0.3,
    min_samples=3
)
plot(Xs, results)

## 3. [Hierarchical Clustering](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html#sklearn.cluster.AgglomerativeClustering)

Hierarchical clustering is a general family of clustering algorithms that build nested clusters by merging or splitting them successively. This hierarchy of clusters is represented as a tree (or __dendrogram__). The root of the tree is the unique cluster that gathers all the samples, the leaves being the clusters with only one sample. See the Wikipedia page for more details.

The __Agglomerative Clustering__ performs a hierarchical clustering using a bottom up approach: each observation starts in its own cluster, and clusters are successively merged together. The linkage criteria determines the metric used for the merge strategy:
- __Ward__ minimizes the sum of squared differences within all clusters. It is a variance-minimizing approach and in this sense is similar to the k-means objective function but tackled with an agglomerative hierarchical approach.
- __Maximum__ or __complete linkage__ minimizes the maximum distance between observations of pairs of clusters.
- __Average linkage__ minimizes the average of the distances between all observations of pairs of clusters.

Agglomerative Clustering can also scale to large number of samples when it is used jointly with a connectivity matrix, but is computationally expensive when no connectivity constraints are added between samples: it considers at each step all the possible merges. - from sklearn's [User Guide](http://scikit-learn.org/stable/modules/clustering.html#hierarchical-clustering).

In [ ]:
from sklearn.cluster import AgglomerativeClustering

- complete

In [ ]:
model, Xs, results = cluster_datasets(
    AgglomerativeClustering,
    preprocess=StandardScaler(),
    n_clusters=3,
    linkage='complete',
)
plot(Xs, results)

In [ ]:
from sklearn.neighbors import kneighbors_graph

In [ ]:
def cluster_connections(**params):
    results = {}
    Xs = {}
    models = {}
    for problem, dataset in datasets.items():
        X, y = dataset
        X = StandardScaler().fit_transform(X, y)
        Xs[problem] = X
        connectivity = kneighbors_graph(X, n_neighbors=10, include_self=False)
        connectivity = 0.5 * (connectivity + connectivity.T)
        model = AgglomerativeClustering(connectivity=connectivity, **params)
        model.fit(X)
        results[problem] = model.labels_.astype(np.int)
        models[problem] = model
    return models, Xs, results

- ward

In [ ]:
models, Xs, results = cluster_connections(
    linkage='ward',
    n_clusters=2,
)
plot(Xs, results)

- average

In [ ]:
models, Xs, results = cluster_connections(
    linkage="average",
    n_clusters=2,
)
plot(Xs, results)

## 4. [Spectral Clustering](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.SpectralClustering.html)

*"Spectral Clustering does a __low-dimension embedding__ of the __affinity matrix__ between samples (similarity matrix), followed by a __K-Means in the low dimensional space__. It is especially efficient if the affinity matrix is sparse. SpectralClustering requires the number of clusters to be specified. It works well for a small number of clusters but is not advised when using many clusters."* - from sklearn's [User Guide](http://scikit-learn.org/stable/modules/clustering.html#spectral-clustering)

In [ ]:
from sklearn.cluster import SpectralClustering

In [ ]:
model, Xs, results = cluster_datasets(
    SpectralClustering,
    preprocess=StandardScaler(),
    n_clusters=2,
    gamma=1e1,
    random_state=42
)
plot(Xs, results)

## 5. [Gaussian Mixture Models](http://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html)

*"A Gaussian mixture model is a probabilistic model that assumes all the data points are generated from a mixture of a finite number of Gaussian distributions with unknown parameters. One can think of mixture models as generalizing k-means clustering to incorporate information about the covariance structure of the data as well as the centers of the latent Gaussians."* - from sklearn's [User Guide](http://scikit-learn.org/stable/modules/mixture.html#gaussian-mixture-models)

A nice tutorial on clustering the iris dataset can be found [here](http://scikit-learn.org/stable/auto_examples/mixture/plot_gmm_covariances.html#sphx-glr-auto-examples-mixture-plot-gmm-covariances-py).

In [ ]:
from sklearn.mixture import GaussianMixture 

In [ ]:
model, Xs, results = cluster_datasets(
    GaussianMixture,
    preprocess=StandardScaler(),
    n_components=3,
    random_state=42
)
plot(Xs, results)

## [Cluster Validation](http://scikit-learn.org/stable/modules/clustering.html#clustering-performance-evaluation)

Evaluating the performance of a clustering algorithm is not as trivial as counting the number of errors or the precision and recall of a supervised classification algorithm. In particular any evaluation metric should not take the absolute values of the cluster labels into account but rather if this clustering define separations of the data similar to some ground truth set of classes or satisfying some assumption such that members belong to the same class are more similar that members of different classes according to some similarity metric.

There are two separate approach to this problem: 

- __Knowing the ground truth__:  
    To test how good a clustering algorithm generally performs, we can apply it to datasets with known labels. This shows how well it will perform on unknown data.
    - [mutual information based scores](http://scikit-learn.org/stable/modules/clustering.html#mutual-information-based-scores): Given the knowledge of the ground truth class assignments `labels_true` and our clustering algorithm assignments of the same samples `labels_pred`, the __Mutual Information__ is a function that measures the agreement of the two assignments, ignoring permutations.
    - [homogeneity, completeness and V-measure](Homogeneity, completeness and V-measure): Given the knowledge of the ground truth class assignments of the samples, it is possible to define some intuitive metric using conditional entropy analysis. Two of such measure is:
        - __homogeneity__: each cluster contains only members of a single class.
        - __completeness__: all members of a given class are assigned to the same cluster
        - their harmonic mean called __V-measure__.

- __Without knowing the ground truth__:  
    Measuring the generated cluster metrics to determine the goodness of the clustering. One example for such a metric is the:
    - [silhouette coefficient](http://scikit-learn.org/stable/modules/clustering.html#silhouette-coefficient) which is defined for each sample and is composed of two scores:
    
        - a: The mean distance between a sample and all other points in the same class.
        - b: The mean distance between a sample and all other points in the next nearest cluster.  
    
    The Silhouette Coefficient $s$ for a single sample is then given as:
    $s = \frac{b - a}{max(a, b)}$


In [ ]:
from sklearn.metrics import silhouette_score

In [ ]:
for problem in datasets.keys():
    print(problem, silhouette_score(Xs[problem], results[problem], random_state=42))